In [1]:
import numpy as np

from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from attention import AttentionSeq2seq2
from ch7.seq2seq import Seq2seq
from ch7.peeky_seq2seq import PeekySeq2seq

In [2]:
# データ読み込み
(x_train, t_train), (x_test, t_test) = sequence.load_data("dataset/date.txt")
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, x_test.shape)

x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

(45000, 29) (5000, 29)


In [3]:
print(x_train.shape, t_train.shape)
type(char_to_id), char_to_id["s"]

(45000, 29) (45000, 11)


(dict, 0)

In [4]:
# ハイパーパラメータ
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5.0

In [5]:
model = AttentionSeq2seq2(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [6]:
%%time
acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose, is_reverse=True)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print("val acc %.3f%%" % (acc*100))

[Trainer] batch_x: (128, 29)
[Trainer] batch_t: (128, 11)
[AttentionDecoder] xs: (128, 10), enc_hs: (128, 29, 256)
out: (128, 10, 16)
dec_hs: (128, 10, 256) <- この長さが理想の10倍
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[AttentionWeight] hs: (128, 29, 256), h: (128, 256)
[TimeAttention.backward] dout: (128, 10, 59)


ValueError: cannot reshape array of size 7552 into shape (128,1,256)

In [7]:
# model.save_params()